In [ ]:
C:\Users\Lawlzee\Documents\Projects\Lawlzee\TAO3\TAO3\TAO3\build.ps1
#!csharp
#i C:\Users\Lawlzee\Documents\Projects\Lawlzee\TAO3\TAO3\TAO3\bin\Feed
#r "nuget:TAO3"
using static TAO3.Prelude;

In [ ]:
#i C:\Users\Lawlzee\Documents\Projects\Lawlzee\TAO3\TAO3\TAO3\bin\Feed
#r "nuget:TAO3"
using static TAO3.Prelude;

Restore sources C:\Users\Lawlzee\Documents\Projects\Lawlzee\TAO3\TAO3\TAO3\bin\Feed

Installed package TAO3 version 1.0.0

Loading extensions from `TAO3.dll`

In [ ]:
#r nuget:System.Data.SqlClient
#i C:\Users\Lawlzee\Documents\Projects\Lawlzee\TAO3\TAO3\TAO3\bin\Feed
#r "nuget:TAO3"

using static TAO3.Prelude;
using System.Data.SqlClient;

string connectionString = "Server= localhost\\SQLEXPRESS; Database=TestDatabase; Integrated Security=True;";
string schemaQuery = @"SELECT [EventType]
,[SchemaName]
,[ObjectName]
,[ObjectType]
,[EventDate]
,[SystemUser]
,[CurrentUser]
,[OriginalUser]
FROM [TAO3].[SchemaChange]";
string queueName = "MyQueue";
SqlDependency.Start(connectionString);

/*
ALTER DATABASE [DBNAME] SET ENABLE_BROKER WITH ROLLBACK IMMEDIATE
CREATE QUEUE myQueue
CREATE SERVICE myService ON QUEUE myQueue 

([http://schemas.microsoft.com/SQL/Notifications/PostQueryNotification])

*/

/*
CREATE SCHEMA TAO3

CREATE TABLE [TAO3].[SchemaChange]
(
	[EventType] [varchar](100) NULL,
	[SchemaName] [varchar](100) NULL,
	[ObjectName] [varchar](100) NULL,
	[ObjectType] [varchar](100) NULL,
	[EventDate] [datetime] NULL,
	[SystemUser] [varchar](100) NULL,
	[CurrentUser] [varchar](100) NULL,
	[OriginalUser] [varchar](100) NULL
)
GO
CREATE TRIGGER TAO3_SchemaChanged_Trigger
ON DATABASE
FOR DDL_DATABASE_LEVEL_EVENTS
AS
set nocount on
declare @EventType varchar(100)
declare @SchemaName varchar(100)
declare @ObjectName varchar(100)
declare @ObjectType varchar(100)
SELECT 
 @EventType = EVENTDATA().value('(/EVENT_INSTANCE/EventType)[1]','nvarchar(max)')  
,@SchemaName = EVENTDATA().value('(/EVENT_INSTANCE/SchemaName)[1]','nvarchar(max)')  
,@ObjectName = EVENTDATA().value('(/EVENT_INSTANCE/ObjectName)[1]','nvarchar(max)')
,@ObjectType = EVENTDATA().value('(/EVENT_INSTANCE/ObjectType)[1]','nvarchar(max)')   
-- Is the default schema used 
if @SchemaName = ' ' select @SchemaName = default_schema_name from sys.sysusers u join sys.database_principals p 
                        on u.uid = p.principal_id  where u.name = CURRENT_USER
insert into [TAO3].[SchemaChange] 
      select @EventType, @SchemaName, @ObjectName, @ObjectType, getdate(), SUSER_SNAME(), CURRENT_USER, ORIGINAL_LOGIN()  
*/

In [ ]:
AddNotification();

void AddNotification()
{
    using (SqlConnection connection = new SqlConnection(connectionString))
    {
        connection.Open();
        using (SqlCommand command = new SqlCommand(schemaQuery, connection))
        {
            SqlDependency dependency = new SqlDependency(command);
        
            dependency.OnChange += (s, e) => 
            {
                Toast.Notify(e.Source.ToString(), e.Info.ToString(), DateTimeOffset.Now);
                if (e.Info != SqlNotificationInfo.Invalid)
                {
                    AddNotification();
                }
                
            };
    
            //command.Connection.Open();
            command.ExecuteNonQuery();
        }
    }
}

In [ ]:
SqlDependency.Stop(connectionString);

In [ ]:
#!in notepad line temp

In [ ]:
#!macro ctrl+shift+1
return new { A = 4 };